# DNF 101: Dynamic neural fields in Lava

## Basic populations and connections

Create populations of leaky integrate-and-fire (LIF) neurons.
The `shape` argument determines the number of neurons and their layout.

In [ ]:
from lava.lib.dnf.population.process import Population

population = Population(shape=20)

Create connections between populations using the `connect` function.
The weights can be specified using a sequence of operations (see below).

In [ ]:
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights

population1 = Population(shape=20)
population2 = Population(shape=20)
connect(population1.s_out, population2.a_in, [Weights(20)])

## Dynamic neural fields (DNF)

### Multi-peak DNF

Create dynamic neural fields (DNFs) that support multiple peaks by using the
`MultiPeakKernel` with local excitation and mid-range inhibition. Use the
`Convolution` operation to apply the kernel.

In [ ]:
from lava.lib.dnf.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.connect import connect

dnf = Population(shape=20)

kernel = MultiPeakKernel(amp_exc=25,
                         width_exc=3,
                         amp_inh=-15,
                         width_inh=6)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

### Selective DNF

Create DNFs that are selective and only create a single peak by using the
`SelectiveKernel` with local excitation and global inhibition.

In [ ]:
from lava.lib.dnf.kernels.kernels import SelectiveKernel

dnf = Population(shape=20)

kernel = SelectiveKernel(amp_exc=18,
                         width_exc=3,
                         global_inh=-15)
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

## Input

### Spike generators

To simulate spike input to a DNF, use a spike generator. The
`GaussPiecewiseStaticInput` creates spikes whose frequency
is determined by a Gaussian function along the neurons of a DNF. Over time,
that Gaussian is piecewise static but may change "between pieces". Each line
specifies the parameters of the Gaussian for a duration of time.

In [ ]:
from lava.lib.dnf.inputs import GaussPiecewiseStaticInput
from lava.lib.dnf.operations.operations import Weights

shape = 15

pattern = [{'max_rate':   0, 'center': 5, 'width': 4, 'duration': 25},
           {'max_rate': 100, 'center': 5, 'width': 4, 'duration': 50},
           {'max_rate':   0, 'center': 5, 'width': 4, 'duration': 25}]
gauss_input = GaussPiecewiseStaticInput(shape=shape,
                                        spike_rate_pattern=pattern)

dnf = Population(shape=shape)

connect(gauss_input.s_out, dnf.a_in, [Weights(20)])


## Running and plotting networks

Wrap your model into a hierarchical process (here, called `Network`) and call
its `run()` method, specifying the number of time steps to run for and the
backend the model should run on. To enable plots, create probes before running
and create plots with the probed data after running.

In [ ]:
from lava.core.generic.process import Process
from lava.core.generic.enums import Backend

class Network(Process):
    def _build(self):
        shape=15

        pattern = [{'max_rate':    0, 'center': 5, 'width': 4, 'duration': 25},
                   {'max_rate': 5000, 'center': 5, 'width': 4, 'duration': 50},
                   {'max_rate':    0, 'center': 5, 'width': 4, 'duration': 25}]
        self.gauss_input = GaussPiecewiseStaticInput(shape=shape,
                                                     spike_rate_pattern=pattern)

        self.dnf = Population(shape=shape)

        kernel = MultiPeakKernel(amp_exc=17,
                                 width_exc=3,
                                 amp_inh=-15,
                                 width_inh=6)
        connect(self.dnf.s_out, self.dnf.a_in, [Instar(kernel)])

        connect(self.gauss_input.s_out, self.dnf.a_in, [OneToOne(20)])

        self.dnf.build_probes()
        self.gauss_input.build_probes()


net = Network()
net.run(num_steps=100, backend=Backend.TF)
net.gauss_input.plot()
net.dnf.plot()

## DNF instabilities

Examples demonstrating different dynamic regimes of DNFs can be found in the
`demos` directory. Here are examples running three such demos for the
detection, selection, and working memory regimes, all in 1D.

### Detection

In [ ]:
from lava.lib.dnf.demos.dnf_basics.multi_peak_hysteresis_1d import Architecture

net = Architecture()
time_steps = 600
net.run(time_steps, backend=Backend.TF)
net.plot(time_steps)

### Selection

In [ ]:
from lava.lib.dnf.demos.dnf_basics.selection_hysteresis_1d import Architecture

net = Architecture()
time_steps = 700
net.run(time_steps, backend=Backend.TF)
net.plot(time_steps)

### Memory

In [ ]:
from lava.lib.dnf.demos.dnf_basics.multi_peak_sustained_1d import Architecture

arch = Architecture()
time_steps = 500
arch.run(time_steps, backend=Backend.TF)
arch.plot(time_steps)

## Higher dimensions

Define DNFs over higher dimensionalities by specifying a `shape` with
multiple entries.

##%

shape = (15, 15)
dnf = Population(shape=shape)

##% md

Inputs and kernels must match the dimensionality of the DNF; specify
parameters such as `width` as vectors rather than scalars.

In [ ]:
pattern = [{'max_rate':   0, 'center': [5, 5], 'width': [4, 4], 'duration': 25},
           {'max_rate': 100, 'center': [5, 5], 'width': [4, 4], 'duration': 50},
           {'max_rate':   0, 'center': [5, 5], 'width': [4, 4], 'duration': 25}]
gauss_input = GaussPiecewiseStaticInput(shape=shape,
                                        spike_rate_pattern=pattern)

kernel = MultiPeakKernel(amp_exc=58,
                         width_exc=[3.8, 3.8],
                         amp_inh=-50,
                         width_inh=[7.5, 7.5])
connections = connect(dnf, dnf, [Instar(kernel)])

connect(gauss_input, dnf, [OneToOne(20)])

#### Example

An example for a 2D DNF with input can be found in the `demos` directory.

In [ ]:
from lava.lib.dnf.demos.dnf_basics.selection_hysteresis_2d import Architecture

net = Architecture()
time_steps = 350
net.run(time_steps, backend=Backend.TF)
net.plot(time_steps)

## Larger architectures

### One-to-one connections
When connecting two DNFs that have the same shape (in terms of neurons and
dimensions), use the operation `OneToOne`. It connects each neuron in the
first DNF to its (single) respective neuron in the second DNF.

In [ ]:
dnf1 = Population(shape=(10,))
dnf2 = Population(shape=(10,))

connect(dnf1, dnf2, [OneToOne(40)])

### Projecting dimensions
When connecting two DNFs with different dimensionality, use the operation
`Projection` and specify how the dimensions should be mapped. Dimensions
that are mapped onto each other must have the same number of neurons.
In the following example, dimension 0 of `dnf_1d` is mapped onto dimension
1 of `dnf_2d`.

In [ ]:
from lava.lib.dnf.operations import Projection

dnf_1d = Population(shape=(10,))
dnf_2d = Population(shape=(20, 10))

connect(dnf_1d, dnf_2d, [OneToOne(40), Projection(dmap={0: 1})])

The full architecture is shown in the `demos` directory.

In [ ]:
from lava.lib.dnf.demos.dnf_basics.projection_1d_to_2d import Architecture

time_steps = 200
net = Architecture(time_steps=time_steps)
net.run(time_steps, backend=Backend.TF)
net.plot(time_steps)